In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
from IPython.display import display, Image
import sklearn
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
import tensorflow as tf

/usr/lib/python2.7/dist-packages/subprocess32.py:472: RuntimeWarning: The _posixsubprocess module is not being used. Child process reliability may suffer if your program uses threads.
  "program uses threads.", RuntimeWarning)


In [2]:
data_size = 1000
simulation_sample_size = 59
gain_size  = 6
with open('rawcorrelation.pickle') as f:
    correlation_data = pickle.load(f)
    dataset = []
    for i in range(len(correlation_data)):
        dataset.append(np.concatenate((correlation_data[i][0],correlation_data[i][1])))
    del correlation_data
with open('rawdata.pickle') as f:
    gain = pickle.load(f)

In [3]:
#batch_size = 128
#number_of_batch = 5
#train_size = batch_size*number_of_batch
simulation_sample_size = 59
gain_size = 6
valid_size = 150
train_size = 700
test_size = 150
train_dataset = np.array(dataset[0:train_size],dtype='float32')
train_label = np.array(gain[0:train_size],dtype='float32')
valid_dataset = np.array(dataset[train_size:train_size+valid_size],dtype='float32')
valid_label = np.array(gain[train_size:train_size+valid_size],dtype='float32')
test_dataset = np.array(dataset[train_size+valid_size:],dtype='float32')
test_label = np.array(gain[train_size+valid_size:],dtype='float32')
print(train_dataset.shape)
print(train_label.shape)
print(valid_dataset.shape)
print(valid_label.shape)
print(test_dataset.shape)
print(test_label.shape)

(700, 118)
(700, 6)
(150, 118)
(150, 6)
(150, 118)
(150, 6)


In [ ]:
graph = tf.Graph()
with graph.as_default():
    #tf_train_dataset = tf.placeholder(tf.float32, (batch_size, simulation_sample_size*2))
    #tf_train_label = tf.placeholder(tf.float32, (batch_size, gain_size))
    tf_train_label = tf.constant(train_label)
    tf_train_dataset = tf.constant(train_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    weights = tf.Variable(tf.truncated_normal([simulation_sample_size*2, gain_size]))
    biases = tf.Variable(tf.zeros([gain_size]))
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.pow((tf_train_label-logits),2))
    optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(loss)
    train_prediction = logits
#    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
#    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [13]:
tf_train_label = tf.constant(train_label)
tf_train_dataset = tf.constant(train_dataset/10000)
simulation_sample_size = 59
gain_size = 6

W = tf.Variable(tf.truncated_normal([simulation_sample_size*2, gain_size]))
b = tf.Variable(tf.zeros([gain_size]))
y = tf.matmul(tf_train_dataset, W) + b

loss = tf.reduce_mean(tf.square(y - tf_train_label))

optimizer = tf.train.GradientDescentOptimizer(0.001)
train = optimizer.minimize(loss)
init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

for step in range(0,50000):
   sess.run(train)
   if step%100==0:
        print(step, sess.run(loss))
print (step, sess.run(W), sess.run(b))

0 83305.08
100 nan
200 nan
300 nan
400 nan
500 nan
600 nan
700 nan
800 nan
900 nan
1000 nan
1100 nan
1200 nan
1300 nan
1400 nan
1500 nan
1600 nan
1700 nan
1800 nan
1900 nan
2000 nan
2100 nan
2200 nan
2300 nan
2400 nan
2500 nan
2600 nan
2700 nan
2800 nan
2900 nan
3000 nan
3100 nan
3200 nan
3300 nan
3400 nan
3500 nan
3600 nan
3700 nan
3800 nan
3900 nan
4000 nan
4100 nan
4200 nan
4300 nan
4400 nan
4500 nan
4600 nan
4700 nan
4800 nan
4900 nan
5000 nan
5100 nan
5200 nan
5300 nan
5400 nan
5500 nan


KeyboardInterrupt: 

In [ ]:
num_steps = 500
batch_size = 128
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print("Initialized")
    offset = 0
    for step in range(num_steps):
        sess.run(optimizer)
        print(step, sess.run(loss))
        print (step, sess.run(weights), sess.run(biases))
#        offset = (step*batch_size)%(train_label.shape[0] - batch_size)
#        batch_dataset = train_dataset[offset:offset+batch_size,:]
#        batch_label = train_label[offset:offset+batch_size,:]
#        feed_dict = {tf_train_dataset : batch_dataset, tf_train_label : batch_label}
#       _,l,predictions = session.run([optimizer,loss, train_prediction], feed_dict = feed_dict)
#        if step%5 == 0:
#            print('Loss at step %d: %f' % (step, l))
    print('End')